In [ ]:
# Proprietary data loading removed.
# df = pd.read_csv('proprietary_dataset.csv')

import pandas as pd
import numpy as np

# Create dummy compound feature matrix
np.random.seed(0)
fake_df = pd.DataFrame({
    'BB1_A': np.random.randint(0, 2, 100),
    'BB1_B': np.random.randint(0, 2, 100),
    'BB2_C': np.random.randint(0, 2, 100),
    'BB3_D': np.random.randint(0, 2, 100),
    'fluorescence': np.random.normal(1000, 100, 100)
})
fake_df.head()

# Same pre-processing approach as the hit identification method

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# intensity delta difference
df['intensity'] = df['81nM'] - df['BL']

#Bead_ID to (row, col) in right-to-left in row-major order
df['row'] = (df['Bead_ID'] - 1) // 200
df['col'] = 199 - ((df['Bead_ID'] - 1) % 200)

# smoothing function for nearest neighbors
def smooth_by_knn(group, k=8):
    coords = group[['row', 'col']].values
    intensities = group['intensity'].values
    n_points = len(coords)
    
    if n_points <= 1:
        return pd.Series(intensities, index=group.index)
    
    n_neighbors = min(k + 1, n_points)  # Include self in query
    nbrs = NearestNeighbors(n_neighbors=n_neighbors).fit(coords)
    _, indices = nbrs.kneighbors(coords)

    #subtract median of neighbors (excluding self)
    local_bg = np.array([
        np.median(intensities[neigh[1:]]) if len(neigh) > 1 else 0
        for neigh in indices
    ])
    corrected = np.maximum(intensities - local_bg, 0)# prevents negatives by clipping at zero
    return pd.Series(corrected, index=group.index)

#spatial smoothing per FOV
df['spatially_corrected'] = df.groupby('FOV', group_keys=False).apply(smooth_by_knn)

df['final_intensity'] = df['spatially_corrected']


# Sampling for a smaller dataset
To create a representative subsample of the data, I performed stratified sampling based on the distribution of log_intensity values. First, I divided the data into five quantile-based bins using pd.qcut, ensuring that each bin contained approximately the same number of entries and captured a specific range of log_intensity. Then, I sampled 20% of rows from each bin, ensuring that the resulting sample preserved the overall distribution of intensity values. After sampling, I dropped the bin column and shuffled the rows to remove any ordering bias. This approach ensures that the subsample is both statistically representative and randomized. I also log transformed the final intensity.

# Log transforming the data

In [ ]:
df['log_final_intensity'] = np.log1p(df['final_intensity'])

#5 quantiles
df['intensity_bin'] = pd.qcut(df['log_final_intensity'], q=5, duplicates='drop')

# stratified sample: 20% from each bin
df_sample = df.groupby('intensity_bin', group_keys=False).sample(frac=0.2, random_state=42)
df_sample = df_sample.drop(columns=['intensity_bin']).sample(frac=1.0, random_state=42).reset_index(drop=True)

print(f"Sampled {len(df_sample):,} rows (stratified by log-transformed final intensity)")


In [ ]:
# ONE HOT ENCODING

encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
X = encoder.fit_transform(df_sample[['BB1', 'BB2', 'BB3']])
y = df_sample['log_final_intensity']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Bayesian optimization using hyperopt (randomized grid searches and normal grid searches took very long)

In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np
from numpy.random import default_rng

subset_idx = np.random.choice(X_train.shape[0], int(0.3 * X_train.shape[0]), replace=False)

X_sub = X_train[subset_idx]
y_sub = y_train.iloc[subset_idx.tolist()]  
space = {
    'n_estimators': hp.quniform('n_estimators', 100, 300, 25),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'subsample': hp.uniform('subsample', 0.7, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1.0),
    'gamma': hp.uniform('gamma', 0, 3),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 0.5),
}

# Objective function to minimize (negative MSE)
def objective(params):
    model = XGBRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        n_jobs=1,
        random_state=42
    )
    score = cross_val_score(model, X_sub, y_sub, scoring='neg_mean_squared_error', cv=3).mean()
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()
# from numpy.random import default_rng
# rstate = default_rng(42)
# Run Hyperopt search
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=25,
    trials=trials,
    rstate = default_rng(42)

)

print("\nBest hyperparameters found:")
print(best)


To help the model learn outliers, I applied sample weighting during training. Specifically, I assigned a weight of 5.0 to molecules in the top 5% of final intensity values (81nM − baseline, spatially corrected), and a weight of 1.0 to all others. This weighting scheme encourages the model to prioritize accurate predictions for top performing binders. By increasing the penalty for errors on these rare, high-intensity examples, the model becomes more sensitive to patterns that distinguish strong binders from the rest.


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
best_params = {
    'colsample_bytree': 0.8897006129500347,
    'gamma': 1.2609857524276593,
    'learning_rate': 0.0881136665371867,
    'max_depth': int(10.0),
    'n_estimators': int(275.0),
    'reg_alpha': 0.13697324747328013,
    'reg_lambda': 0.14798705569405798,
    'subsample': 0.9948938425189908,
    'n_jobs': -1,
    'random_state': 42
}


threshold = np.percentile(y_train, 95)
#give 5× weight to top 5% examples, 1× to the rest
sample_weight = np.where(y_train >= threshold, 5.0, 1.0)


final_model = XGBRegressor(**best_params)
final_model.fit(X_train, y_train,sample_weight=sample_weight)

y_pred = final_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nFinal model performance:")
print(f"Test MSE: {mse:.6f}")


In [ ]:
import joblib

joblib.dump(final_model, 'final_xgb_model.pkl')

final_model = joblib.load('final_xgb_model.pkl')
joblib.dump(encoder, 'final_encoder.pkl')

# Later
encoder = joblib.load('final_encoder.pkl')


In [ ]:
top_5_cutoff = y_test.quantile(0.95)
high_idx = y_test >= top_5_cutoff

y_pred_high = final_model.predict(X_test[high_idx])
from sklearn.metrics import mean_squared_error, r2_score

print("Top-5% RMSE:", mean_squared_error(y_test[high_idx], y_pred_high, squared=False))


# Discussion

The final model achieved a test MSE of 0.863 on log-transformed intensity values, indicating an average prediction error of about 0.93 in log-space. Since the model predicts log(final_intensity + 1), this corresponds to a 2-fold error in the original intensity scale. this result definitely suggetss potential for improvement with model tuning, or maybe pre-processing the features differently. For outliers, the RMSE was even higher at 1.86 in the log space. Overall this model needs a lot more refinement.

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test[high_idx], y_pred_high, alpha=0.3)
plt.xlabel("True log intensity (top 5%)")
plt.ylabel("Predicted log intensity")
plt.title("Predicted vs True for Top 5% Molecules")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.grid(True)
plt.show()


# Model does worse on outliers (top 5%)

## Top 100 Molecules in the dataset itself

In [ ]:
import pandas as pd
import numpy as np
import joblib
from IPython.display import display

final_model = joblib.load('final_xgb_model.pkl')
encoder = joblib.load('final_encoder.pkl')

df_sample["molecule"] = df_sample["BB1"] + "_" + df_sample["BB2"] + "_" + df_sample["BB3"]
seen_combos = df_sample[['BB1', 'BB2', 'BB3', 'molecule']].drop_duplicates().reset_index(drop=True)

# Predict on seen combinations
X_seen = encoder.transform(seen_combos[['BB1', 'BB2', 'BB3']])
preds = final_model.predict(X_seen)
seen_combos["predicted_log_intensity"] = preds

top_100_seen = seen_combos.sort_values("predicted_log_intensity", ascending=False).head(100)

print("Top 100 Seen Molecules by Predicted Intensity:")
display(top_100_seen)

top_100_seen.to_csv("top_100_seen_molecules.csv", index=False)


## Unseen molecules (new combinations)

In [ ]:
import pandas as pd
import numpy as np
import itertools
import joblib
from IPython.display import display

final_model = joblib.load('final_xgb_model.pkl')
encoder = joblib.load('final_encoder.pkl')

# Generate all possible BB1-BB2-BB3 combinations from the full dataset
bb1_unique = df['BB1'].unique()
bb2_unique = df['BB2'].unique()
bb3_unique = df['BB3'].unique()

all_combos = pd.DataFrame(
    list(itertools.product(bb1_unique, bb2_unique, bb3_unique)),
    columns=["BB1", "BB2", "BB3"]
)


df_sample["molecule"] = df_sample["BB1"] + "_" + df_sample["BB2"] + "_" + df_sample["BB3"]
seen_molecules = set(df_sample["molecule"])

all_combos["molecule"] = (
    all_combos["BB1"] + "_" +
    all_combos["BB2"] + "_" +
    all_combos["BB3"]
)

# Filter out seen molecules
unseen_combos = all_combos[~all_combos["molecule"].isin(seen_molecules)].reset_index(drop=True)

X_unseen = encoder.transform(unseen_combos[['BB1', 'BB2', 'BB3']])
unseen_preds = final_model.predict(X_unseen)
unseen_combos["predicted_log_intensity"] = unseen_preds

top_100_unseen = unseen_combos.sort_values("predicted_log_intensity", ascending=False).head(100)

print("Top 100 Unseen Molecules by Predicted Intensity:")
display(top_100_unseen)

top_100_unseen.to_csv("top_100_unseen_molecules.csv", index=False)


# Discussion
Upon reviewing the top 100 predicted molecules, the model appears to prioritize similar features to those identified during hit calling. Notably, BB3 components such as CP_80, CP_194, and CP_95 frequently appear among the highest-scoring predictions. This overlap suggests that the model has successfully learned some signal patterns consistent with my hit calling method.

Below is other scratch work I did not end up using.

In [ ]:
# from sklearn.linear_model import Ridge

# model = Ridge(alpha=1.0, random_state=42)
# model.fit(X_train, y_train)
# from sklearn.metrics import mean_squared_error, r2_score

# y_pred = model.predict(X_test)
# print("MSE:", mean_squared_error(y_test, y_pred))
# print("R²:", r2_score(y_test, y_pred))


In [ ]:
# from xgboost import XGBRegressor
# from sklearn.metrics import mean_squared_error, r2_score

# xgb_model = XGBRegressor(
#     n_estimators=200,
#     max_depth=12,
#     learning_rate=0.1,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     n_jobs=-1
# )

# xgb_model.fit(X_train, y_train)


In [ ]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

space = {
    'n_estimators': hp.quniform('n_estimators', 100, 300, 25),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'subsample': hp.uniform('subsample', 0.7, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1.0),
    'gamma': hp.uniform('gamma', 0, 3),
    'reg_alpha': hp.uniform('reg_alpha', 0, 0.5),
    'reg_lambda': hp.uniform('reg_lambda', 0, 0.5),
}

# Objective function to minimize (neg MSE)
def objective(params):
    model = XGBRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        n_jobs=1,              y        random_state=42
    )
    score = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=3).mean()
    return {'loss': -score, 'status': STATUS_OK}

trials = Trials()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=25,             
    trials=trials,
    rstate=np.random.RandomState(42)
)

print("\nBest hyperparameters found:")
print(best)


In [ ]:
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost MSE: {mse:.2f}")
print(f"XGBoost R²: {r2:.6f}")


In [ ]:
# Sample 30% of the training set
X_df = pd.DataFrame(X_train)         # in case X_train is still ndarray
y_series = pd.Series(y_train)        # same for y_train

X_sub = X_df.sample(frac=0.3, random_state=42)
y_sub = y_series.iloc[X_sub.index]   # use iloc here


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import uniform, randint

# Define hyperparameter grid
param_dist = {
    'n_estimators': randint(100, 600),
    'max_depth': randint(5, 15),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': uniform(0, 5),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1),
}


xgb = XGBRegressor(random_state=42, n_jobs=1)


random_search = RandomizedSearchCV(
    xgb,
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1,
    random_state=42,
    n_jobs=1 
)

# Fit
# Perform hyperparameter tuning with CV on this subset
random_search.fit(X_sub, y_sub)


In [ ]:

best_xgb = random_search.best_estimator_

y_pred = best_xgb.predict(X_test)
print(f"MSE: {mean_squared_error(y_test, y_pred):.4f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest MSE: {mse_rf:.6f}")
print(f"Random Forest R²: {r2_rf:.3f}")


In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

svr_model = make_pipeline(
    StandardScaler(with_mean=False),  # don't center sparse matrices
    SVR(kernel='rbf', C=1.0, epsilon=0.1)
)

svr_model.fit(X_train, y_train)
y_pred_svr = svr_model.predict(X_test)


mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print(f"SVR MSE: {mse_svr:.6f}")
print(f"SVR R²: {r2_svr:.3f}")
